In [1]:
from pybaseball.lahman import *
import pandas as pd
import matplotlib.pylab as plt

class yearAvg:
    average = {}
    weights = {}

    def __init__(self, year):
        self.setAvgs(year)
        self.setWeight()

    def setAvgs(self, year):
        bat = batting()
        bat = bat.dropna()
        bat = bat.loc[(bat['yearID']==year) & (bat['AB']>0)]
        bat['BA'] = bat['H']/bat['AB']
        bat['ISO'] = (bat['2B']+2*bat['3B']+3*bat['HR'])/bat['AB']
        self.average['BA'] = bat['BA'].mean() # hits/AB - young
        self.average['ISO'] = bat['ISO'].mean() # young
        self.average['BB'] = bat['BB'].mean() # bases on balls (walks) - old
        self.average['SO'] = bat['SO'].mean() # strike outs - old
        self.average['SB'] = bat['SB'].mean() # stolen bases - young
        self.average['SF'] = bat['SF'].mean() # sacfriced flies - old

    def setWeight(self):
        for i in self.average:
            self.weights[i] = (4.20 / self.average[i]) # I want my scores to be avergeing around 4.20

    def getAvgs(self):
        return self.average
    def getWeight(self):
        return self.weights

In [30]:
class player:
    playerID = ''
    playerName = ''
    palyerBirthYear = 0
    yearsPlayed = []
    ChapoScore = 0
    yearISO = 0
    data = pd.DataFrame()


    def __init__(self, playerID):
        self.playerID = playerID
        self.setData(self.playerID)
        self.setInfo(self.playerID)

    def getStatcast(self, playerID):
        return dp.DataFrame()

    def setInfo(self, playerID):
        try:
            masterData = master()
            playerData = masterData.loc[masterData['playerID']==playerID].set_index('playerID')
            self.playerName = playerData['nameFirst'][playerID] + ' ' + playerData['nameLast'][playerID]
            self.palyerBirthYear = playerData['birthYear'][playerID]
            self.yearsPlayed = self.data['yearID'].tolist()
        except:
            print("Failed to find player")

    def getName(self):
        return self.playerName

    def setData(self, playerID):
        bat = batting()
        bat = bat.loc[bat['playerID']==playerID]
        bat['BA'] = bat['H']/bat['AB']
        bat['ISO'] = (bat['2B']+2*bat['3B']+3*bat['HR'])/bat['AB']
        self.data = bat[['playerID','yearID','BA','ISO','SO','BB','SB','SF']]

    def getData(self):
        return self.data

    def setChapoScore(self, year):
        # get the year averages for the used scores
        yearAverages = yearAvg(year)
        ya = yearAverages.getAvgs()
        # get all the data for one player in a specific year
        yearData = self.data.loc[self.data['yearID'] == year].set_index('playerID')
        age = yearData['yearID'][self.playerID] - self.palyerBirthYear
        age_weight = 0.00625*(float(age)-23)**3 + 1
        old_scores = yearData['SF'][self.playerID]/ya['SF']+yearData['BB'][self.playerID]/ya['BB']
        young_scores = yearData['ISO'][self.playerID]/ya['ISO'] + yearData['SB'][self.playerID]/ya['SB']
        self.yearISO = yearData['ISO'][self.playerID]
        try:
            self.ChapoScore = (young_scores*(age_weight) + old_scores*(2-age_weight))/4
        except:
            self.ChapoScore = 0

    def getYearsPlayed(self):
        return self.yearsPlayed
    def getChapoScore(self):
        return self.ChapoScore

    def getSummary(self):
        print("="*30)
        print(self.playerName, "Score Summary")
        print("="*30)
        for year in self.yearsPlayed:
            age = year - self.palyerBirthYear
            self.setChapoScore(year)
            print("age: ",age, "|",year,"score: ",self.getChapoScore(), "| ISO:",self.yearISO)

In [31]:
print("")
playerID = input("Enter a players playerID: ")
print("")
# some good players
  # arenano01
  # harpebr03
# some bad players
  # zimmejo02
  # gennesc01
OldP = player(playerID)
OldP.getSummary()
print("")




Enter a players playerID: zimmejo02

Jordan Zimmermann Score Summary
age:  23.0 | 2009 score:  0.015136013480982185 | ISO: 0.0
age:  24.0 | 2010 score:  0.0 | ISO: 0.0
age:  25.0 | 2011 score:  0.09362714732862831 | ISO: 0.023255813953488372
age:  26.0 | 2012 score:  0.33368672042324965 | ISO: 0.08771929824561403
age:  27.0 | 2013 score:  0.19584420139025072 | ISO: 0.015384615384615385
age:  28.0 | 2014 score:  0.14458348050736242 | ISO: 0.01818181818181818
age:  29.0 | 2015 score:  0.09403773091858433 | ISO: 0.015873015873015872
age:  30.0 | 2016 score:  0.0 | ISO: 0.0

